# quantum gates

In [1]:
using LinearAlgebra

First, let's define some useful functions to improve readability

In [2]:
# convert the Kronecker product into an operator instead of a function
⊗ = kron

# replace small numbers by zero to
sparsefy(m) = round.(m; digits=2)

# visualise in terms of 0s and 1s
roundint(m) = round.(Int,m)

# avoid hide rows and columns specially in complex matrices
fulldisplay(x) = show(IOContext(stdout, :limit=>false), MIME"text/plain"(), x)

# some gates combinations differ from the standard implementation only by a global phase
# in order to guarantee we have the right gate we can add the extra phase manually. The
# negative sign here is to facilitate the connection between code and mathematical notation.
global_phase(θ) = exp(-im * θ);

## Primitive gates

In [3]:
id = Matrix(I,2,2)
xgate = [0 1; 1 0]
ygate = [0 -im; im 0]
zgate = [1 0; 0 -1]

2×2 Matrix{Int64}:
 1   0
 0  -1

### two-qubits gates
These gates can actually be implemented in terms of each other so, it's enough that one of than is available as a native operation. Usually that means the $Z \otimes Z$ interaction since most of physical system falls either in the Ising model
$$H_{ising} = - \sum_n \omega_n \sigma_n^z - \sum_{i,j} J_{ij} \sigma_i^z \sigma_j^z$$
or in the Heisenberg model
$$H_{heisenberg} = - \sum_n \omega_n \sigma_n^z - \sum_n J_n^x \sigma_n^x \sigma_{n+1}^x - \sum_n J_n^y \sigma_n^y \sigma_{n+1}^y - \sum_n J_n^z \sigma_n^z \sigma_{n+1}^z$$


In [4]:
zzgate = zgate ⊗ zgate

czgate = [1 0 0 0;
          0 1 0 0;
          0 0 1 0;
          0 0 0 -1]

cnotgate = [1 0 0 0;
            0 1 0 0;
            0 0 0 1;
            0 0 1 0]

swapgate = [1 0 0 0;
            0 0 1 0;
            0 1 0 0;
            0 0 0 1];

## Primitive rotations
From the primitive gates we can implement the basic local rotation that are actually used as gates in physicial implmentations.

In [5]:
rx(θ) = exp(-im * θ/2 * xgate)
ry(θ) = exp(-im * θ/2 * ygate)
rz(θ) = exp(-im * θ/2 * zgate)

# As said before, the Z⊗Z interaction may be a primitive gate so we add it as a primitive rotation
rzz(θ) = exp(-im * θ/2 * zzgate);

Note all the rotations a defined with the half of the angle, $e^{-i\frac{\theta}{2} G}$, as in the standard form for ½-spin evolution.

## Factorising single-qubits gates

### Hadamard
The standard Hadamard gate
$$\mathrm{H} = \frac{1}{\sqrt{2}} \begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix}$$
is not a primitive gate. However, using the Bloch sphere, is easy to see it as a $\frac{\pi}{2}$ rotation around $Y$-axis.

In [6]:
hadamard = ry(π/2)

2×2 Matrix{ComplexF64}:
 0.707107+0.0im  -0.707107+0.0im
 0.707107+0.0im   0.707107+0.0im

This rotation moves $|1\rangle \to -|-\rangle$. In pratical terms this extra phase, usually, is irrelevante but we can add a $\pi$ rotation around $Z$ direction to correct the phase.

In [7]:
hadamard = ry(π/2) * rz(π)

hadamard |> sparsefy

2×2 Matrix{ComplexF64}:
 0.0-0.71im  -0.0-0.71im
 0.0-0.71im   0.0+0.71im

Now the only difference to the standard Hadamard is a $\frac{\pi}{2}$ phase.

In [8]:
hadamard = global_phase(-π/2) * ry(π/2) * rz(π)

hadamard |> real

2×2 Matrix{Float64}:
 0.707107   0.707107
 0.707107  -0.707107

In mathematical terms we have
$$\mathrm{H} = \frac{1}{\sqrt{2}} \,\begin{bmatrix}
1 & 1 \\
1 & -1
\end{bmatrix} = e^{-i\frac{\pi}{2}} R_Y (\pi/2) R_Z (\pi) .$$

## Factorising two-qubits gates

### $C_{not}$ as a $C_Z$ interaction
This is a standard version since any controlled operation can be generated by a controlled-Z.

In [9]:
cnot_cz = (id ⊗ ry(π/2)) * czgate * (id ⊗ ry(-π/2))

cnot_cz |> real |> roundint 

4×4 Matrix{Int64}:
 1  0  0  0
 0  1  0  0
 0  0  0  1
 0  0  1  0

$$C_{not} = \big[ \mathbb{I} \otimes R_Y(\pi/2) \big] \,
C_Z \,
\big[ \mathbb{I} \otimes R_Y^\dagger(\pi/2) \big] .$$

Note that the $R_Y$ goes on the target qubit.

### $C_{not}$ as a $R_{ZZ}$ interaction
A common implementation in NMR combines a $R_{ZZ}$ with two local operations in $X$ and $Y$ direction. The basic version is the following

In [10]:
cnot_zz = (id ⊗ ry(-π/2)) * rzz(π/2) * (id ⊗ rx(π/2))

cnot_zz |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
 0.71-0.71im   0.0-0.0im     0.0+0.0im    0.0+0.0im
 -0.0-0.0im   0.71+0.71im    0.0+0.0im    0.0+0.0im
  0.0+0.0im    0.0+0.0im     0.0+0.0im   0.71-0.71im
  0.0+0.0im    0.0+0.0im   -0.71-0.71im   0.0-0.0im

Although it doesn't look like to standard Cnot gate, it can be seem as a Cnot followed by two local $R_Z$ rotations in opposite direction.

In [11]:
cnot_zz = (id ⊗ ry(-π/2)) * rzz(π/2) * (id ⊗ rx(π/2)) * (rz(π/2) ⊗ rz(-π/2))

cnot_zz |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
 0.71-0.71im  -0.0-0.0im    0.0+0.0im    0.0+0.0im
 -0.0-0.0im   0.71-0.71im   0.0+0.0im    0.0+0.0im
  0.0+0.0im    0.0+0.0im   -0.0+0.0im   0.71-0.71im
  0.0+0.0im    0.0+0.0im   0.71-0.71im   0.0-0.0im

This is exactly a cnot with a $\frac{\pi}{4}$ phase. In that case the full expresion would be like

In [12]:
cnot_zz = global_phase(-π/4) * (id ⊗ ry(-π/2)) * rzz(π/2) * (id ⊗ rx(π/2)) * (rz(π/2) ⊗ rz(-π/2))

cnot_zz |> real |> roundint 

4×4 Matrix{Int64}:
 1  0  0  0
 0  1  0  0
 0  0  0  1
 0  0  1  0

$$C_{not} = e^{-i\frac{\pi}{4}}
\big[ \mathbb{I} \otimes R_Y^\dagger(\pi/2) \big] \,
R_{ZZ}(\pi/2) \,
\big[ \mathbb{I} \otimes R_X(\pi/2) \big] \,
\big[ R_Z(\pi/2) \otimes R_Z^\dagger(\pi/2) \big] .$$

### $C_Z$ as a $R_{ZZ}$
In essence a $C_Z$ interaction can be seem as a $R_{ZZ}$ followed local $R_Z$ rotations in the same diretion. This construction only works for $\frac{\pi}{4}$ rotations

In [13]:
cz_zz = rzz(π/2) * (rz(-π/2) ⊗ rz(-π/2))

cz_zz |> fulldisplay

4×4 Matrix{ComplexF64}:
 0.707107+0.707107im       0.0+0.0im            0.0+0.0im             0.0+0.0im
      0.0+0.0im       0.707107+0.707107im       0.0+0.0im             0.0+0.0im
      0.0+0.0im            0.0+0.0im       0.707107+0.707107im        0.0+0.0im
      0.0+0.0im            0.0+0.0im            0.0+0.0im       -0.707107-0.707107im

Eliminating the $\frac{\pi}{4}$ phase we recover the $C_{ZZ}$ gate.

In [14]:
cz_zz = global_phase(π/4) * rzz(π/2) * (rz(-π/2) ⊗ rz(-π/2))

cz_zz |> real |> roundint 

4×4 Matrix{Int64}:
 1  0  0   0
 0  1  0   0
 0  0  1   0
 0  0  0  -1

$$C_Z = e^{i\frac{\pi}{4}} \, R_{ZZ}(\pi/2) \,
\big[ R_Z^\dagger (\pi/2) \otimes R_Z^\dagger (\pi/2) \big] .$$

### $R_{XX}$, $R_{YY}$, $R_{XY}$, and $R_{YX}$ in terms of $R_{ZZ}$
Since we can easily move $Z$ to $X$ or $Y$,
$$\mathrm{H}Z \mathrm{H} = X ,$$
$$S\mathrm{H}Z \mathrm{H}S = Y ,$$
we can algebricaly perfom the simple transformations
$$\mathrm{H} e^{i \theta Z} \mathrm{H} = e^{i \theta X} ,$$
$$S\mathrm{H} e^{i \theta Z} \mathrm{H}S = e^{i \theta Y} .$$

However, as said above in the section about single gates, the Hadamard (and the phase gate as well) can be used as simple rotantion around $Y$ (and $X$ respectively),
$$\big[ R_Y(\pi/2) \otimes R_Y(\pi/2) \big] \,
e^{i \theta Z} \,
\big[ R_Y^\dagger(\pi/2) \otimes R_Y^\dagger(\pi/2) \big] \to e^{i \theta X} ,$$
$$\big[ R_X^\dagger(\pi/2) \otimes R_X^\dagger(\pi/2) \big] \,
e^{i \theta Z} \,
\big[ R_X(\pi/2) \otimes R_X(\pi/2) \big] \to e^{i \theta Y} .$$

In [15]:
rxx(θ) = (ry(π/2) ⊗ ry(π/2)) * rzz(θ) * (ry(-π/2) ⊗ ry(-π/2))

ryy(θ) = (rx(-π/2) ⊗ rx(-π/2)) * rzz(θ) * (rx(π/2) ⊗ rx(π/2));

Using the same principle we can create mixed rotarions $R_{XY}$ and $R_{YX}$.

In [16]:
rxy(θ) = (ry(π/2) ⊗ rx(-π/2)) * rzz(θ) * (ry(-π/2) ⊗ rx(π/2))

ryx(θ) = (rx(-π/2) ⊗ ry(π/2)) * rzz(θ) * (rx(π/2) ⊗ ry(-π/2));

### Swap gate as $R_{Z}$
Algebraically the swap gate can be written as
$$Swap =
\frac{1}{2} \left( \mathbb{I} + \vec{S}_1 \cdot \vec{S}_2 \right) =
\frac{1}{2} \left( \mathbb{I} + X_1 X_2 + Y_1 Y_2 + Z_1 Z_2  \right) .$$

When the $C_{not}$ can be found as a primitive operation, the swap gate can be written as three alternated $C_{not}$ gates similar to the swap operation using XOR ports in classical computers.

An alternative construction can be found using $R_{ZZ}$ interactions.

In [17]:
swap_zz = (ry(π/2) ⊗ ry(π/2)) * rzz(π/2) * (ry(-π/2) ⊗ ry(-π/2)) *    # R_xx
          (rx(-π/2) ⊗ rx(-π/2)) * rzz(π/2) * (rx(π/2) ⊗ rx(π/2)) *    # R_yy
          rzz(π/2)

swap_zz |> sparsefy|> fulldisplay

4×4 Matrix{ComplexF64}:
 0.71-0.71im  -0.0-0.0im   -0.0-0.0im    0.0+0.0im
 -0.0-0.0im    0.0+0.0im   0.71-0.71im   0.0-0.0im
 -0.0-0.0im   0.71-0.71im   0.0+0.0im    0.0-0.0im
  0.0-0.0im   -0.0+0.0im   -0.0+0.0im   0.71-0.71im

which differs from the standard gate by a $\frac{\pi}{4}$ phase.

In [18]:
swap_zz = global_phase(-π/4) *
          rxx(π/2) *
          ryy(π/2) *
          rzz(π/2)

swap_zz |> real |> roundint 

4×4 Matrix{Int64}:
 1  0  0  0
 0  0  1  0
 0  1  0  0
 0  0  0  1

In simple terms,
$$Swap = e^{-i\frac{\pi}{4}} \, R_{XX}(\pi/2) \, R_{YY}(\pi/2) \, R_{ZZ}(\pi/2) .$$

## Interactions using $\sigma^+$ and $\sigma^-$ using $R_{ZZ}$

### $\sigma^+ \sigma^- + \sigma^- \sigma^+$ (flip-flop)
Since
$$\frac{\sigma^+ \sigma^- + \sigma^- \sigma^+}{2} = XX + YY ,$$

In [19]:
flipflop_zz(θ) = rxx(θ) * ryy(θ)
                
flipflop_zz(π/3) |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
  1.0-0.0im  -0.0-0.0im   -0.0-0.0im   0.0+0.0im
  0.0-0.0im   0.5-0.0im    0.0-0.87im  0.0+0.0im
  0.0-0.0im   0.0-0.87im   0.5-0.0im   0.0+0.0im
 -0.0-0.0im  -0.0+0.0im   -0.0+0.0im   1.0-0.0im

### $\sigma^+ \sigma^+ + \sigma^- \sigma^-$ (sync)
Since
$$\frac{\sigma^+ \sigma^+ + \sigma^- \sigma^-}{2} = XX - YY ,$$

In [20]:
sync_zz(θ) = rxx(θ) * ryy(-θ)
                
sync_zz(π/3) |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
  0.5-0.0im   0.0-0.0im  0.0-0.0im   0.0-0.87im
 -0.0-0.0im   1.0+0.0im  0.0+0.0im  -0.0+0.0im
 -0.0-0.0im   0.0+0.0im  1.0+0.0im  -0.0+0.0im
 -0.0-0.87im  0.0+0.0im  0.0+0.0im   0.5+0.0im

### $i\left(\sigma^+ \sigma^- - \sigma^- \sigma^+ \right)$ (asymmetric flip-flop)
Since
$$i \frac{\sigma^+ \sigma^- - \sigma^- \sigma^+}{2} = XY - YX ,$$

In [21]:
asym_flipflop_zz(θ) = rxy(θ) * ryx(-θ)

asym_flipflop_zz(π/3) |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
  1.0+0.0im   -0.0+0.0im   0.0-0.0im   0.0+0.0im
  0.0+0.0im    0.5+0.0im  0.87+0.0im  -0.0+0.0im
 -0.0-0.0im  -0.87+0.0im   0.5-0.0im   0.0-0.0im
  0.0-0.0im    0.0+0.0im  -0.0-0.0im   1.0-0.0im

### $i\left(\sigma^+ \sigma^+ - \sigma^- \sigma^- \right)$ (asymmetric sync)
Since
$$i \frac{\sigma^+ \sigma^+ - \sigma^- \sigma^-}{2} = XY + YX ,$$

In [22]:
asym_sync_zz(θ) = rxy(θ) * ryx(θ)
                
asym_sync_zz(π/3) |> sparsefy |> fulldisplay

4×4 Matrix{ComplexF64}:
  0.5+0.0im  -0.0-0.0im  -0.0-0.0im  -0.87+0.0im
  0.0-0.0im   1.0+0.0im  -0.0-0.0im    0.0+0.0im
  0.0-0.0im  -0.0+0.0im   1.0+0.0im    0.0+0.0im
 0.87-0.0im  -0.0+0.0im  -0.0+0.0im    0.5+0.0im

## References
_**TODO...**_